# Introduction and Initialization

In [1]:
import os
import math
import warnings
import numpy as np
warnings.simplefilter('ignore')

from scipy.signal import argrelmin # to find minima of potentials to extract true minimum 
from mpi4py import MPI

import eptl
import eptl.create_lammps_pairstyle_table as clpt # functions to create and test tabulated pair potential
clpt.set_plot_rcparams()

# Set your lammps executable
#from eptl.create_lammps_pairstyle_table import LmpExec
#Lmp = LmpExec()
#Lmp.change_exec("/path/to/your/lammps/executable/lmp")

print("eptl: v", eptl.__version__)

# Write Settings
folderToUse = "tables/"
os.system("mkdir -p "+folderToUse)

lmps_input_filename = folderToUse+"pair_rep_coh_smooth_linear_scaled_{}.in"
plot_filename = folderToUse+"comparison_coh_smooth.png"

pair_keyword1 = "rep_coh_smooth_linear"
pair_filename1 = folderToUse + "Table_pair_rep_coh_smooth_linear_temp_{}.txt"
lmps_input_filename1 = folderToUse + "pair_rep_coh_smooth_linear_{}.in"

pair_keyword2 = "rep_coh_smooth_linear_scaled"
pair_filename2 = folderToUse + "Table_pair_rep_coh_smooth_linear_scaled_{}.txt"
lmps_pair_filename2 = folderToUse + "Table_lmps_pair_rep_coh_smooth_linear_scaled{}.txt"

pair_filename_diff = folderToUse + "Table_pair_rep_coh_smooth_diff_linear_scaled_{}.txt"

# Set-up MPI
comm = MPI.COMM_WORLD
nprocs = comm.Get_size()
rank = comm.Get_rank()

# LAMMPS Settings
units_string = "lj"

# Pair-Style Parameters
epr = 20.0 # "kT"
sigma = 1.0
rc = 1.5

# Pair-Style Settings
dist_mode = 2
rmin = 0.5 # minimum distance
rmax = 2.0 # maximum distance
N = 3000 # number of distance values between rmin and rmax to use (note N+1 points get printed!)
res = 5 # resolution (# of decimal places) to round minene, this is should be at least one order of mag lower than O(rmin)*O(N^-1)

rdelta = ( rmax - rmin ) / N
rlist = np.arange( rmin, rmax, rdelta)
rctab = rlist[np.where( rlist < rc)[0][-1]]

## RANGE OF EPCs ##
epc_delta = 1.0
epcmax = 16.0
epcmin = 0.0

rank_epc_split = ( epcmax - epcmin ) / nprocs

rank_epcmax = rank_epc_split * (rank + 1) + epcmin
rank_epcbegin = rank_epc_split * rank + epcmin
epcs = np.arange( rank_epcbegin, rank_epcmax, epc_delta)

eptl: v v2.2.0


# Iterate Through $\epsilon$ Values

In [3]:
for epc in epcs:

    epc = np.around(epc,1)
    # make the initial (pre-scaled) table
    clpt.make_table_for_lammps(
        pair_filename1.format(rank),
        pair_keyword1, 
        rmin=rmin, 
        rmax=rmax, 
        N=N, 
        args=(epr, epc, sigma, rc, rctab),
        mode=dist_mode,
    )
    
    # grab true minimum from the table
    minene = 0.0
    if epc > 0.0:
        pairdata = np.genfromtxt( pair_filename1.format(rank), skip_header=5)

        r = pairdata[:,1]
        ene = pairdata[:,2]

        min_index = argrelmin(ene)[0][0]
        assert(min_index >= 0)

        minene = np.around( ene[min_index], res)
        minr = np.around( r[min_index], res)
        if dist_mode == 2:
            minr = np.around( math.sqrt( r[min_index]), res)
    
    else: # The repulsive case is trivial
        minene = np.around( 0.0, res) 
        minr = np.around( sigma*2**(1./6.), res)

    scale_fact = 1.0
    if epc > 0.0:
        scale_fact = -epc/minene
        
    # generate a label for the pair_table with specific parameters
    label = f"_sigma-{sigma}_minr-{minr}_rc-{rc}_epr-{epr}_epc-{epc}_rmin-{rmin}_rmax-{rmax}_N-{N}"

    # Make Re-scaled Table
    clpt.make_table_for_lammps(
        pair_filename2.format(label),
        pair_keyword2,
        rmin, 
        rmax, 
        N,
        args=(scale_fact, epr, epc, sigma, rc, rctab),
        mode=dist_mode
    )

    # Get lammps to pair_write the potential energy and force for comparison
    clpt.pair_write_lammps(
        lmps_input_filename.format(label),
        lmps_pair_filename2.format(label),
        pair_filename2.format(label), 
        units_string,
        rc=rc,
        mode=dist_mode
    )
    # Compare the data in files visually
    clpt.comparison(
        pair_filename_diff.format(label),
        lmps_pair_filename2.format(label),
        pair_filename2.format(label),
        rlim=(rmin,rmax),
        plim=(-epc-10,100),
        flim=(-4.0*epc,30),
        markers=False,
        mode=dist_mode
    ) 
    
    print(f"(MPI-rank-{rank}) DONE: {lmps_pair_filename2.format(label)}")

# [ for the above clpt.comparison()] turn plot=True to plot=False to only output file for relative differences between lammps and your original table. In the above test_pair_LJ_rel_differences.txt contains all the numrical difference data and test_pair_LJ.pdf is the plot of the potentials and forces. So the first argument is <some-string> which will make <some-string>_rel_differences.txt and <some-string>.pdf




(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.12246_rc-1.5_epr-20.0_epc-0.0_rmin-0.5_rmax-2.0_N-3000.txt
(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.14837_rc-1.5_epr-20.0_epc-1.0_rmin-0.5_rmax-2.0_N-3000.txt
(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.14837_rc-1.5_epr-20.0_epc-2.0_rmin-0.5_rmax-2.0_N-3000.txt
(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.14837_rc-1.5_epr-20.0_epc-3.0_rmin-0.5_rmax-2.0_N-3000.txt
(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.14837_rc-1.5_epr-20.0_epc-4.0_rmin-0.5_rmax-2.0_N-3000.txt
(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.14837_rc-1.5_epr-20.0_epc-5.0_rmin-0.5_rmax-2.0_N-3000.txt
(MPI-rank-0) DONE: tables/Table_lmps_pair_rep_coh_smooth_linear_scaled_sigma-1.0_minr-1.14837_rc-1.5_epr-20.0_epc-6.0_